This code will start with some of the data cleaning that was suggested by the first exploration

Note that we'll need to install the agcounts package in the conda distribution.

This can be done from the terminal. First, activate the distribution of conda we're using:
conda activate erdos_fall_2024

Then install:
conda install agcounts

(Note - I had some trouble getting this to work. I think I had to restart the conda environment to get it to work...)

In [9]:
# Start by importing packages we'll need
import pandas as pd
import numpy as np
from agcounts.extract import get_counts
import plotly.express as px
import plotly.graph_objects as go

In [16]:
# Load the parquet data file series_train.parquet/id=0a418b57/part-0.parquet
data0a418b57 = pd.read_parquet('series_train.parquet/id=0a418b57/part-0.parquet')

In [17]:
# Add a new column that converts time_of_day into datetime
data0a418b57['dt'] = pd.to_datetime(data0a418b57['time_of_day'])

# Change the day in the dt variable to be equal to the relative_date_PCIAT value
data0a418b57['dt_mod'] = data0a418b57['dt'] + pd.to_timedelta(data0a418b57['relative_date_PCIAT'], unit='D')

In [18]:
# Modify the code from the readme file in the agcounts GitHub to do something here

# Note, for our data:
# df will be data0a418b57, but we could replace this with a read_parquet command
#   also, the function assumes that the X, Y, and Z variables are called "X","Y", and "Z"
# freq = ??????
# epoch = 5 seconds... I think????
# time_column = dt_mod
# fast = ???

def get_counts_parquet(
    df,
    freq: int,
    epoch: int,
    fast: bool = True,
    verbose: bool = True,
    time_column: str = None,
):
    if verbose:
        print("Reading in Parquet", flush=True)
    raw = df
    if time_column is not None:
        ts = raw[time_column]
        ts = pd.to_datetime(ts)
        time_freq = str(epoch) + "S"
        ts = ts.dt.round(time_freq)
        ts = ts.unique()
        ts = pd.DataFrame(ts, columns=[time_column])
    raw = raw[["X", "Y", "Z"]]
    if verbose:
        print("Converting to array", flush=True)
    raw = np.array(raw)
    if verbose:
        print("Getting Counts", flush=True)
    counts = get_counts(raw, freq=freq, epoch=epoch, fast=fast)
    del raw
    counts = pd.DataFrame(counts, columns=["Axis1", "Axis2", "Axis3"])
    counts["AC"] = (
        counts["Axis1"] ** 2 + counts["Axis2"] ** 2 + counts["Axis3"] ** 2
    ) ** 0.5
    ts = ts[0 : counts.shape[0]]
    if time_column is not None:
        counts = pd.concat([ts, counts], axis=1)
    return counts


def convert_counts_parquet(
    file,
    outfile,
    freq: int,
    epoch: int,
    fast: bool = True,
    verbose: bool = False,
    time_column: str = None,
):
    counts = get_counts_parquet(
        file, freq=80, epoch=5, verbose=True, time_column=time_column
    )
    counts.to_csv(outfile, index=False)
    return counts


counts = get_counts_parquet(data0a418b57, freq=80, epoch=5, fast = True, verbose = True, time_column = 'dt_mod')

# I don't think we need to export this to a file?
#counts = convert_counts_parquet(
#    data0a418b57,
#    outfile="counts.csv.gz",
#    freq=80,
#    epoch=5,
#    verbose=True,
#    time_column="dt_mod",
#)

Reading in Parquet
Converting to array
Getting Counts


/var/folders/9z/6ssrlr3d4ln53k8ljbbv93p00000gn/T/ipykernel_19446/1054894727.py:26: FutureWarning:

'S' is deprecated and will be removed in a future version, please use 's' instead.



In [19]:
# Take a look at the data generated by the previous algorithm. Going to look at the tail...
counts.tail()

,dt_mod,Axis1,Axis2,Axis3,AC
528,1969-12-23 17:39:00,0,0,0,0.000000
529,1969-12-23 17:39:05,159,207,158,305.113094
530,1969-12-23 17:39:10,119,267,624,689.076193
531,1969-12-23 17:39:15,266,395,898,1016.457082
532,1969-12-23 17:39:20,359,664,865,1148.042682


It looks like the counts only looked at 533 rows from the original data. Need to try to fix that...

Let's compare the AC values to the enmo values

In [10]:
# Make a line plot with plotly from the data0a418b57 data frame with "dt_mod" as the x-variable, and "enmo" as the y-variable. Restrict data to relative_date_PCIAT = -9
fig = go.Figure()
fig.add_trace(go.Scatter(x=data0a418b57[data0a418b57['relative_date_PCIAT'] == -9]['dt_mod'], y=data0a418b57[data0a418b57['relative_date_PCIAT'] == -9]['enmo'], mode='lines', name='enmo', line=dict(color='blue', width=2)))
fig.update_layout(title='ENMO data over time for relative_date_PCIAT = -9', xaxis_title='Time', yaxis_title='ENMO', showlegend=True)
fig.show()

In [13]:
# Make a line plot with plotly from the counts data frame with "dt_mod" as the x-variable, and "AC" as the y-variable.
fig = go.Figure()
fig.add_trace(go.Scatter(x=counts['dt_mod'], y=counts['AC'], mode='lines', name='AC', line=dict(color='red', width=2)))
fig.update_layout(title='AC data over time', xaxis_title='Time', yaxis_title='AC', showlegend=True)
fig.show()